### Compute # of Amenity "Points of Interests" near each exit

In this notebook, we extract OpenStreetMap (OSM) amenity locations near our remaining candidate sites. This involves buffering the area around all exit features and using that buffered shape to extract OSM "point of interest" (POI) data using the `osmnx` Python package. 

The "Automating GIS" [Lesson 6](https://automating-gis-processes.github.io/site/notebooks/L6/retrieve_osm_data.html#Points-of-interest) provides a useful overview of this process. The [documentation on the `osmnx`](https://osmnx.readthedocs.io/en/stable/) provides more information and is quite useful for exploring more about what the package can do.

An outline of the workflow:
* Import the remaining candidate sites (i.e. exits) into our code environment
* Buffer the exit point features 1 mile
 * This requires ensuring our features using a projected crs
* Transforming the buffered features to the WGS 84 (epsg:4326) crs (to match OSM data)
* Combining the set of buffered exit features to a single multipolygon geometry (for OSM query)
* Querying OSM amenity points of interest found within the above geometry
* OPTIONAL: save the resulting features to a CSV
* OPTIONAL: save the resulting features to a shapefile
 * Requires ensuring that all feature geometry types are the same
* Iterating through each exit feature and counting the number of amenities within 1 mile
* Appending the count of amenities tallied above to the exits attribute table; save as shapefile

In [ ]:
#Import packages
import geopandas as gpd
import osmnx as ox
import shapely

#import shapely.speedups
from shapely import speedups
speedups.enabled

In [ ]:
#Read in exits feature class as a geodataframe
theExits = gpd.read_file('../Data/processed/Exits_distance_to_DCFC.shp')
type(theExits)

* As we are going to buffer our features, we need to ensure the features have a projected coordinate reference system. If not, we should transform the data so that it does.

In [ ]:
#Check the spatial reference of the exit features
theExits.crs

We see that our dataset uses the UTM Zone 17N (NAD83) coordinate reference system. It's units are in meters, so we are set to buffer the data.

*TIP: you could also peek at the values in the `geometry` column. If the numbers are relatively small (e.g. between 0 and 180) your data most likely uses a geographic crs (with values in degrees). However, if the values are in the hundreds of thousands, the data are most likley in meters or feet.*

* Next we need to buffer the features. We'll choose 1 mile as the distance around each exit we'd like to search for amenities. After buffering the features, we'll need to transform our features to the WGS84 coordinate reference system, as that is what the `osmnx` package requires for spatially selecting OSM data. And finally, we'll need to combine all our set of individidual exit buffers into a single shape - using the shapely `unary_union` function. This again is to meet the requirements of the `osmnx` package.

In [ ]:
#Buffer the exits 1 mile (1609.34 m); transform to WGS84, and dissolve into a single shape
theExits['geometry'] = theExits['geometry'].buffer(1609.34)

In [ ]:
#Transform the buffered features to WGS 84
theExits_wgs84 = theExits.to_crs(4326)

In [ ]:
#Combine features into a single multipolygon
theSearchArea = theExits_wgs84.unary_union
type(theSearchArea)

* And now we use the `osmnx` packages's `create_poi_gdf()` function to extract amenities within the search area geometry. Here, we'll pull restaurantes and cafes. See the [OSM documentation](https://wiki.openstreetmap.org/wiki/Key:amenity) for what other amenities we can extract.<br>*Note: this take a bit of time...*

In [ ]:
#Extract restaurants in search area 
theAmenities = ox.pois.create_poi_gdf(polygon=theSearchArea,amenities=['restaurant','cafes'])

In [ ]:
#Save the data to a CSV
theAmenities.to_csv('../Data/OSM/Amenities_1mile.csv')

To save our data as a shapefile, we need to ensure that the data contains only one "feature type", e.g., just points *or* polygons, as shapefiles can't store multiple types. So, let's examine the variety of feature types in our geodataframe of amenities.

In [ ]:
#List the unique types of features
theAmenities['geometry'].type.unique()

We see we have three types of shapes: `point`, `polygon`, and `multipolygon`. We can either save each into separate shapefiles, or we can convert them all to the same feature type, i.e. a point (by converting polygons and multipolygons to points via their centroids).
* Both choices begin with splitting the geodataframe into separate dataframes, one for points and one for polygons/multipolygons. 

In [ ]:
#Parse the dataframe to three, based on geometry type
theAmenities_points = theAmenities.loc[theAmenities['geometry'].type == 'Point'].reset_index()
theAmenities_polys = theAmenities.loc[theAmenities['geometry'].type != 'Polygon'].reset_index()

* We could export each into separate shapefiles, but instead we'll convert the Polygon and MultiPolygon features to Points by taking their centroids.

In [ ]:
#Convert polygon features to points by taking their centroids
theAmenities_polys['geometry'] = theAmenities_polys['geometry'].centroid
theAmenities_polys['geometry'].type.unique()

* And now that all three are point features, let's merge them together and export as a shapefile

In [ ]:
#Append the three together
import pandas as pd
theAmenities_all = pd.concat([theAmenities_points,theAmenities_polys])
theAmenities_all.shape

In [ ]:
#Save to a shapefile, selecting only the amenity type as an attribute
theAmenities_all[['amenity','geometry']].to_file('../Data/OSM/OSM_amenities.shp')

In [ ]:
#Get the area around one exit feature (the 56th one, just to grab one I know has amenities)
theArea = theExits_wgs84.at[56,'geometry']

In [ ]:
#Create a mask of amenities within this shape
theSpatialMask = theAmenities_all.within(theArea)

In [ ]:
#Count returns the number of "trues" within this mask
theSpatialMask.sum()

There are _2_ amenities within 1 mile of the 56th exit feature.

Putting this together, we construct a function that returns the county of amenitry features within the buffer area. Then we can `apply` this function to the geometry series in the buffered exits dataframe, returning a series containing the full set of amenity counts - to which we can append to the exits attribute table. 

In [ ]:
#Define a function
def get_amenity_count(theShape):
    return theAmenities_all.within(theShape).sum()

In [ ]:
#Apply the function to the get
amenity_count = theExits_wgs84['geometry'].apply(get_amenity_count)

In [ ]:
#Plot a histogram of the counts
amenity_count.hist()

In [ ]:
#Re-read in the exit features (since we modified them) and attach the amenity count data
theExits = gpd.read_file('../Data/processed/Exits_distance_to_DCFC.shp')
theExits['amenity_n'] = amenity_count
theExits.head()

In [ ]:
#Finally, save the exits to a new feature class
theExits.to_file('../Data/processed/exits_amenities.shp')